# sample code

In [1]:
import numpy as np
import stan
import pandas as pd
import nest_asyncio, asyncio
nest_asyncio.apply()

schools_code = """
data {
  int<lower=0> J;         // number of schools
  array[J] real y;              // estimated treatment effects
  array[J] real<lower=0> sigma; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data, random_seed=1)


Building...



Building: found in cache, done.Messages from stanc:
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.


In [2]:
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()
df
theta_cols = [c for c in df.columns if c.startswith("theta.")]
df[theta_cols].mean()
df[theta_cols].quantile([0.025, 0.5, 0.975])

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 7.5e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.75 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 6.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.68 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.000135 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.35 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 0.0001 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1 seconds.
  Adjust your expectations accordingly!


parameters,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6,theta.7,theta.8
0.025,-2.260854,-4.730981,-11.158860,-6.151228,-8.470961,-8.748435,-1.343254,-6.933209
0.500,10.112686,7.847120,6.564002,7.479713,5.620318,6.425592,10.001497,8.136686
0.975,31.847154,20.681862,19.855098,21.170041,15.910623,18.646668,26.684652,26.012415


In [3]:
poisson_hier_code = """
data {
  int<lower=1> N;                         // number of observations
  int<lower=1> J;                         // number of groups
  array[N] int<lower=0> y;                // counts
  array[N] int<lower=1, upper=J> g;       // group index for each obs (1..J)
  vector[N] x;                             // predictor
}
parameters {
  real alpha;                              // global intercept
  real beta;                               // slope for x
  real<lower=0> tau;                       // sd of group effects
  vector[J] z;                             // non-centered group effects ~ N(0,1)
}
transformed parameters {
  vector[J] u = tau * z;                   // group effects
  vector[N] log_lambda;
  for (n in 1:N) {
    log_lambda[n] = alpha + beta * x[n] + u[g[n]];
  }
}
model {
  // priors
  alpha ~ normal(0, 5);
  beta  ~ normal(0, 5);
  tau   ~ exponential(1);
  z     ~ normal(0, 1);

  // likelihood (Poisson with log link)
  y ~ poisson_log(log_lambda);
}
"""


In [4]:
rng = np.random.default_rng(123)
J = 8                       # number of groups
N = 400                     # number of observations
g = rng.integers(1, J+1, size=N)   
x = rng.normal(size=N)             

alpha_true = 0.5
beta_true  = 1.2
tau_true   = 0.7
u_true = rng.normal(0, tau_true, size=J)

eta = alpha_true + beta_true * x + u_true[g-1]
lam = np.exp(eta)
y = rng.poisson(lam)

data = {
    "N": N,
    "J": J,
    "y": y.tolist(),          
    "g": g.tolist(),         
    "x": x.tolist(),          
}

posterior = stan.build(poisson_hier_code, data=data, random_seed=1)
fit = posterior.sample(num_chains=4, num_samples=1000)

df = fit.to_frame()
print(df.head())

for name in ["alpha", "beta", "tau"]:
    s = df[name].describe(percentiles=[0.025, 0.5, 0.975])
    print(f"\n{name} posterior summary:\n{s}")


Building...



Building: found in cache, done.Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  31% (2500/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling:  40% (3200/8000)
Sampling:  41% (3300/8000)
Sampling:  43% (3401/8000)
Sa

parameters         lp__  accept_stat__  stepsize__  treedepth__  n_leapfrog__  \
draws                                                                           
0           2536.286225       0.998347    0.056416          4.0          31.0   
1           2536.531511       0.932158    0.055203          7.0         143.0   
2           2540.466526       0.951352    0.063265          6.0         127.0   
3           2539.557376       0.965701    0.061203          3.0           7.0   
4           2538.901879       0.993710    0.056416          7.0         127.0   

parameters  divergent__     energy__     alpha      beta       tau  ...  \
draws                                                               ...   
0                   0.0 -2529.639783  1.181418  1.151719  1.099871  ...   
1                   0.0 -2531.117061  1.335153  1.205519  0.911959  ...   
2                   0.0 -2531.338105  0.811385  1.214047  0.833510  ...   
3                   0.0 -2537.412869  0.484823  1.176821 

In [6]:
df = fit.to_frame()
theta_cols = [c for c in df.columns if c.startswith("alpha")]
df[theta_cols].mean()
df[theta_cols].quantile([0.025, 0.5, 0.975])


parameters,alpha
0.025,0.031070
0.500,0.788506
0.975,1.533539


# Real data
RPL13 gene_cell; RPL13 gRNA_cell

In [ ]:
import stan

poisson_glm_single_alpha_code = """
data {
  int<lower=1> N;                 // number of cells
  array[N] int<lower=0> y;        // gene A counts
  vector[N] x;                    // known log1p(gRNA) per cell
  // optional: vector[N] offset;  // known log-offset (e.g., log library size)
}
parameters {
  real beta0;                     // intercept
  real beta;                     // global perturbation strength (slope on x)
  real<lower=0> sigma_eps;        // natural biological variation
  real<lower=0> sigma_del;        // perturbation strength variability, extra biological effects
  vector[N] eps_raw;              // non-centered residuals eps
  vector[N] del_raw;              // non-centered residuals del
}
transformed parameters {
  vector[N] eps = sigma_eps*eps_raw;
  vector[N] del = sigma_del*del_raw;
  vector[N] alpha = beta*x + del;
  vector[N] eta = beta0 + alpha + eps;  // optional: + offset 
}
model {
  // priors (weakly informative)
  beta0      ~ normal(0, 5);
  beta      ~ normal(0, 5);
  sigma_eps  ~ exponential(1);
  sigma_del  ~ exponential(1);
  
  // residuals
  eps_raw ~ normal(0, 1);
  del_raw ~ normal(0, 1);
  
  // likelihood (Poisson-lognormal)
  y ~ poisson_log(eta);  // use y ~ poisson_log(eta + offset) if you include offset
}
generated quantities {
  vector[N] lambda = exp(eta);    // optional, for PPC or summaries
}
"""


In [ ]:
import numpy as np
import scipy.sparse as sp
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

readRDS = ro.r['readRDS']
Matrix  = importr('Matrix')  # ensures Matrix classes are known

def read_rds_auto(path):
    r_obj = readRDS(path)
    r_class = list(ro.r['class'](r_obj))
    print(f"{path} -> R class: {r_class}")

    # dgCMatrix (column-compressed sparse); convert to SciPy csc_matrix
    if "dgCMatrix" in r_class:
        i   = np.array(r_obj.slots['i'],  dtype=np.int32)
        p   = np.array(r_obj.slots['p'],  dtype=np.int32)
        x   = np.array(r_obj.slots['x'],  dtype=float)
        dim = tuple(np.array(r_obj.slots['Dim'], dtype=np.int32))
        mat = sp.csc_matrix((x, i, p), shape=dim)
        return mat

    # dense matrix -> numpy
    if "matrix" in r_class:
        return np.array(r_obj)

    # data.frame -> pandas
    from rpy2.robjects import pandas2ri
    with ro.conversion.localconverter(ro.default_converter + pandas2ri.converter):
        try:
            import pandas as pd  # just to be sure pandas is present
            return ro.conversion.rpy2py(r_obj)
        except Exception:
            pass

    # Unknown type: return the raw R object (you can handle case-by-case)
    return r_obj

counts = read_rds_auto("/net/wonderland/home/zhaixt/fibro_aging_jin/data/counts_sub.rds")
gmat   = read_rds_auto("/net/wonderland/home/zhaixt/fibro_aging_jin/data/gmat_sub.rds")

print(type(counts), getattr(counts, 'shape', None))
print(type(gmat),   getattr(gmat,   'shape', None))


/net/wonderland/home/zhaixt/fibro_aging_jin/data/counts_sub.rds -> R class: ['dgCMatrix']
/net/wonderland/home/zhaixt/fibro_aging_jin/data/gmat_sub.rds -> R class: ['dgCMatrix']
<class 'scipy.sparse._csc.csc_matrix'> (1, 253259)
<class 'scipy.sparse._csc.csc_matrix'> (4, 253259)


In [28]:
gmat_total = gmat.sum(axis=0).A1
gmat_log1p = np.log1p(gmat_total)

In [55]:
idx = np.flatnonzero(gmat_total > 0)
gmat_total_filter = gmat_total[idx]
gmat_log1p_filter = np.log1p(gmat_total_filter)


In [57]:
counts_dense = counts.toarray()
counts_filter = counts_dense[:, idx]

In [62]:
data = {"N": len(counts_filter[0]), "y": counts_filter[0].astype(int), "x": gmat_log1p_filter.astype(float)}

In [64]:
len(counts_filter[0])

3978

In [70]:

posterior = stan.build(poisson_glm_single_alpha_code, data=data, random_seed=1)
fit = posterior.sample(num_chains=4, num_samples=1000)

# α is a scalar → one posterior distribution (length = chains*samples)
alpha_draws = fit["alpha"]                    # shape: (num_draws,)
alpha_mean  = alpha_draws.mean()
alpha_se    = alpha_draws.std(ddof=1)
alpha_ci    = np.quantile(alpha_draws, [0.025, 0.5, 0.975])

print("alpha mean:", alpha_mean)
print("alpha se:", alpha_se)
print("alpha 95% CrI:", alpha_ci)


Building...

In file included from /net/wonderland/home/zhaixt/.local/share/r-miniconda/lib/python3.11/site-packages/httpstan/include/tbb/concurrent_unordered_map.h:26,
                 from /net/wonderland/home/zhaixt/.local/share/r-miniconda/lib/python3.11/site-packages/httpstan/include/stan/math/rev/core/profiling.hpp:10,
                 from /net/wonderland/home/zhaixt/.local/share/r-miniconda/lib/python3.11/site-packages/httpstan/include/stan/math/rev/core.hpp:53,
                 from /net/wonderland/home/zhaixt/.local/share/r-miniconda/lib/python3.11/site-packages/httpstan/include/stan/math/rev.hpp:10,
                 from /net/wonderland/home/zhaixt/.local/share/r-miniconda/lib/python3.11/site-packages/httpstan/include/stan/math.hpp:19,
                 from /net/wonderland/home/zhaixt/.local/share/r-miniconda/lib/python3.11/site-packages/httpstan/include/stan/model/model_header.hpp:4,
                 from /net/wonderland/home/zhaixt/.cache/httpstan/4.13.0/models/t23bsrx2/model_t23bsrx2.


Building: 32.8s, done.Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2401/8000)
Sampling:  31% (2501/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling:  40% (3200/8000)
Sampling:  

alpha mean: -0.7972240196498543
alpha se: 1.10087058584319
alpha 95% CrI: [-3.8246641  -0.3167125   0.32170073]


In [71]:
df = fit.to_frame()
df

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,beta0,beta,sigma_eps,...,lambda.3969,lambda.3970,lambda.3971,lambda.3972,lambda.3973,lambda.3974,lambda.3975,lambda.3976,lambda.3977,lambda.3978
draws,,,,,,,,,,,,,,,,,,,,,
0,479562.171582,0.857975,0.087839,6.0,63.0,0.0,-475556.076207,4.122794,-0.383165,0.704725,...,41.043444,32.607627,32.749065,43.864518,61.336701,1.694627,54.908148,77.428035,65.821067,81.503390
1,479514.528189,0.814932,0.059896,6.0,63.0,0.0,-475425.242548,4.109127,-0.377746,0.094215,...,29.322691,40.556967,19.179033,44.040396,71.540821,1.375863,62.941514,86.759755,62.435956,78.743780
2,479636.150868,0.999024,0.031622,8.0,255.0,0.0,-475726.444164,4.118800,-0.395231,0.215332,...,35.990464,45.728797,21.675020,51.395863,63.223200,2.803792,56.277750,96.537399,71.852316,69.983188
3,479618.168913,0.969913,0.101404,5.0,31.0,0.0,-475613.495803,4.119117,-0.383181,0.722599,...,47.034949,35.634597,20.203142,43.268573,66.112453,1.750173,79.468168,81.989089,57.851167,77.644806
4,479596.178928,0.910137,0.087839,6.0,63.0,0.0,-475626.713202,4.122127,-0.385398,0.710479,...,37.332767,44.264783,15.174826,47.140180,71.514038,0.804081,73.270100,90.187444,67.458870,57.783625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,479626.198721,0.792893,0.101404,5.0,31.0,0.0,-475593.266071,4.100740,-0.376061,0.705415,...,52.268518,41.252528,25.948779,43.718086,57.473149,1.476619,76.943532,89.347528,60.003967,59.872580
3996,479531.782623,0.232960,0.087839,6.0,63.0,0.0,-475540.100245,4.106230,-0.381383,0.718128,...,39.341125,26.564396,23.519118,38.652114,69.054259,2.113163,65.624815,90.292100,60.835601,64.856801
3997,479612.676387,0.534206,0.059896,7.0,127.0,0.0,-475676.286344,4.119637,-0.381410,0.186360,...,46.760673,28.284155,21.854788,37.681312,72.010231,1.007789,71.656950,73.966202,71.608373,62.855880
